本代码参考论文引用网络节点分类比赛 Baseline：https://aistudio.baidu.com/aistudio/projectdetail/1199841

## 代码整体逻辑

1. 读取提供的数据集，包含构图以及读取节点特征（用户可自己改动边的构造方式）

2. 配置化生成模型，用户也可以根据教程进行图神经网络的实现。

3. 开始训练

4. 执行预测并产生结果文件


In [1]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [2]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd

## 图网络配置

这里已经有很多强大的模型配置了，你可以尝试简单的改一下config的字段。
例如，换成GAT的配置
```
config = {
    "model_name": "GAT",
    "num_layers":  1,
    "dropout": 0.5,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "edge_dropout": 0.00,
}
```

In [3]:
from easydict import EasyDict as edict

config = {
    "model_name": "GCN",
    "num_layers": 1,
    "dropout": 0.5,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "edge_dropout": 0.00,
}

config = edict(config)

## 数据加载模块

这里主要是用于读取数据集，包括读取图数据构图，以及训练集的划分。

In [4]:
from collections import namedtuple

Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("work/feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})
    
    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    
    df = pd.read_csv("work/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_index) * 0.8)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("work/test.csv")["nid"].values
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    return dataset

In [5]:
dataset = load()

train_index = dataset.train_index
train_label = np.reshape(dataset.train_label, [-1 , 1])
train_index = np.expand_dims(train_index, -1)

val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")


## 组网模块

这里是组网模块，目前已经提供了一些预定义的模型，包括**GCN**, **GAT**, **APPNP**等。可以通过简单的配置，设定模型的层数，hidden_size等。你也可以深入到model.py里面，去奇思妙想，写自己的图神经网络。

In [6]:
import pgl
import model
import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model

# 使用CPU
place = fluid.CPUPlace()

# 使用GPU
# place = fluid.CUDAPlace(0)

train_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()
with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        gw, loss, acc, pred = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)

test_program = fluid.Program()
with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        _gw, v_loss, v_acc, v_pred = build_model(dataset,
            config=config,
            phase="test",
            main_prog=test_program)


test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)

## 开始训练过程

图神经网络采用FullBatch的训练方式，每一步训练就会把所有整张图训练样本全部训练一遍。



In [7]:
epoch = 200
exe.run(startup_program)

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
feed_dict = gw.to_feed(dataset.graph)

for epoch in range(epoch):
    # Full Batch 训练
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(train_index, dtype="int64")
    feed_dict["node_label"] = np.array(train_label, dtype="int64")
    
    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)

    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0])

Epoch 0 Train Acc 0.17270592 Valid Acc 0.18900833
Epoch 1 Train Acc 0.18203175 Valid Acc 0.19043212
Epoch 2 Train Acc 0.20922618 Valid Acc 0.21299921
Epoch 3 Train Acc 0.2240692 Valid Acc 0.24482095
Epoch 4 Train Acc 0.25172636 Valid Acc 0.28525662
Epoch 5 Train Acc 0.275824 Valid Acc 0.32569233
Epoch 6 Train Acc 0.308678 Valid Acc 0.38214564
Epoch 7 Train Acc 0.3340393 Valid Acc 0.41161814
Epoch 8 Train Acc 0.34398803 Valid Acc 0.39794973
Epoch 9 Train Acc 0.35165873 Valid Acc 0.36705345
Epoch 10 Train Acc 0.34806365 Valid Acc 0.3498256
Epoch 11 Train Acc 0.34009042 Valid Acc 0.34448636
Epoch 12 Train Acc 0.34140742 Valid Acc 0.3468356
Epoch 13 Train Acc 0.3449491 Valid Acc 0.35281554
Epoch 14 Train Acc 0.34893572 Valid Acc 0.36014807
Epoch 15 Train Acc 0.35922262 Valid Acc 0.36506015
Epoch 16 Train Acc 0.36530933 Valid Acc 0.3695451
Epoch 17 Train Acc 0.36595002 Valid Acc 0.37417242
Epoch 18 Train Acc 0.37116465 Valid Acc 0.3783014
Epoch 19 Train Acc 0.37735817 Valid Acc 0.38470846
E

Epoch 164 Train Acc 0.6509931 Valid Acc 0.67993164
Epoch 165 Train Acc 0.6492134 Valid Acc 0.6797181
Epoch 166 Train Acc 0.65170497 Valid Acc 0.6777248
Epoch 167 Train Acc 0.65134907 Valid Acc 0.6784367
Epoch 168 Train Acc 0.650566 Valid Acc 0.67808074
Epoch 169 Train Acc 0.6482345 Valid Acc 0.68114185
Epoch 170 Train Acc 0.6502278 Valid Acc 0.6802876
Epoch 171 Train Acc 0.65054816 Valid Acc 0.6782943
Epoch 172 Train Acc 0.65017444 Valid Acc 0.67850786
Epoch 173 Train Acc 0.6500142 Valid Acc 0.68007404
Epoch 174 Train Acc 0.6513134 Valid Acc 0.68114185
Epoch 175 Train Acc 0.65392965 Valid Acc 0.6802876
Epoch 176 Train Acc 0.65216774 Valid Acc 0.68050116
Epoch 177 Train Acc 0.65017444 Valid Acc 0.6796469
Epoch 178 Train Acc 0.6528084 Valid Acc 0.6792198
Epoch 179 Train Acc 0.6515092 Valid Acc 0.68000287
Epoch 180 Train Acc 0.65008545 Valid Acc 0.68035877
Epoch 181 Train Acc 0.65248805 Valid Acc 0.6817114
Epoch 182 Train Acc 0.6544102 Valid Acc 0.68199617
Epoch 183 Train Acc 0.65254146 V

## 对测试集进行预测

训练完成后，我们对测试集进行预测。预测的时候，由于不知道测试集合的标签，我们随意给一些测试label。最终我们获得测试数据的预测结果。


In [10]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

## 生成提交文件

最后一步，我们可以使用pandas轻松生成提交文件，最后下载 submission.csv 提交就好了。

In [11]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission.csv", index=False)